In [22]:
import io
import urllib

import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from tensorflow.keras.preprocessing import image

import numpy as np
import pandas as pd

import cv2

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import tensorflow_hub as hub
from urllib.request import urlopen

import re
import pytesseract
from PIL import Image

from tensorflow.keras.models import load_model

import colorgram
from matplotlib.colors import to_hex

import matplotlib as mpl
import webcolors

from os import listdir
from os.path import isfile, join

import wget


In [23]:
%config InlineBackend.figure_format='retina' 

In [24]:
def load_image_from_url(url, target_size=None, color_mode='rgb'):
    assert color_mode in ('grayscale', 'rgb'), 'color_mode must be "grayscale" or "rgb"'
    response = urllib.request.urlopen(url)
    img = Image.open(io.BytesIO(response.read()))
    img = img.convert('RGB')
    if color_mode == 'grayscale':
        img = ImageOps.grayscale(img)
    if target_size:
        img = img.resize(target_size, Image.NEAREST) # resize
    return image.img_to_array(img)

def load_image_from_path(image_path, target_size=None, color_mode='rgb'):
    pil_image = image.load_img(image_path, 
                               target_size=target_size,
                            color_mode=color_mode)
    return image.img_to_array(pil_image)

In [25]:
# import a list of all movie poster file names.
mypath = 'movie_posters/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [26]:
# import the csv file with data from IMDB
imdb = pd.read_csv('MovieGenre.csv', encoding="ISO-8859-1")

In [27]:
imdb.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [28]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40108 entries, 0 to 40107
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   imdbId      40108 non-null  int64  
 1   Imdb Link   40108 non-null  object 
 2   Title       40108 non-null  object 
 3   IMDB Score  40060 non-null  float64
 4   Genre       39963 non-null  object 
 5   Poster      39383 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.8+ MB


In [29]:
# initialize other parameters:
INPUT_SHAPE = (224, 224)

url_classes = 'https://raw.githubusercontent.com/nightrome/cocostuff/master/labels.txt'
coco_classes = [str(x)[str(x).find(' '):].strip().replace("'", '')
                for x in urlopen(url_classes).read().splitlines()]
hub_model = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1") # load the model only once!

# face classification
model_url = 'https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml'
face_model = wget.download(model_url)

face_classification = cv2.CascadeClassifier(face_model) # load the classifier only once!

def apply_offsets(face_coordinates, offsets):
    """
    Derived from https://github.com/oarriaga/face_classification/blob/
    b861d21b0e76ca5514cdeb5b56a689b7318584f4/src/utils/inference.py#L21
    """
    x, y, width, height = face_coordinates
    x_off, y_off = offsets
    return (x - x_off, x + width + x_off, y - y_off, y + height + y_off)

gender_classifier = load_model('gender_mini_XCEPTION.21-0.95.hdf5') # load this only once! (not in a loop)
GENDER_OFFSETS = (10, 10)
INPUT_SHAPE_GENDER = gender_classifier.input_shape[1:3]

# emotion classification
emotion_classifier = load_model('fer2013_mini_XCEPTION.102-0.66.hdf5') # load this only once! (not in a loop)

EMOTION_OFFSETS = (0, 0)
INPUT_SHAPE_EMOTION = emotion_classifier.input_shape[1:3]

emo_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Definition for color names
def get_colour_name(rgb_triplet):
    """
    From https://stackoverflow.com/questions/9694165/convert-rgb-color-to-english-color-name-like-green-with-python
    """
    min_colours = {}
    for key, name in webcolors.CSS21_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb_triplet[0]) ** 2
        gd = (g_c - rgb_triplet[1]) ** 2
        bd = (b_c - rgb_triplet[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

In [30]:
results = [] # initialize the results dataframe

for i in onlyfiles:
    try: # some files don't seem to work, so I added this, to let the for loop just run further while a error occurs
        # get the info from imdb, it will be stored at the end of the loop (title, genre and rating)
        imdbID = int(i.split('.')[0])
        imdb_info = imdb.loc[imdb['imdbId'] == imdbID]

        # Get the amount of faces
        pre_image = load_image_from_path(''.join(('movie_posters/',i)), target_size=(INPUT_SHAPE), color_mode='grayscale')
        gray_image = np.squeeze(pre_image).astype('uint8')
        faces = face_classification.detectMultiScale(gray_image, 1.3, 5) # detect the faces 
        n_faces = len(faces) # get the number of faces

        # main gender of detected faces:
        gender = 'unknown'

        if n_faces > 0: 
            genders = np.zeros(shape=(n_faces,2)) # initialize
            for j, face_coordinates in enumerate(faces): # using the output of the CascadeClassifier
                x1, x2, y1, y2 = apply_offsets(face_coordinates, GENDER_OFFSETS) # extends the bounding box
                face_img = gray_image[y1:y2, x1:x2] # only get the face 
                face_img = cv2.resize(face_img, (INPUT_SHAPE_GENDER)) # resize the image
                face_img = face_img.astype('float32') / 255.0 # preprocess the image
                face_img = np.expand_dims(face_img, 0) # batch of one
                probas = gender_classifier.predict(face_img) 
                genders[j] = probas
            if np.mean(genders[:,0])>np.mean(genders[:,1]):
                gender = 'woman'
            else:
                gender = 'man'  

        # main emotion of detected faces:
        emotion = 'unknown'

        if n_faces > 0:
            emotions = np.zeros(shape=(n_faces,7))

            for j, face_coordinates in enumerate(faces):
                x1, x2, y1, y2 = apply_offsets(face_coordinates, EMOTION_OFFSETS) 
                face_img = gray_image[y1:y2, x1:x2] # only get the face
                face_img = cv2.resize(face_img, (INPUT_SHAPE_EMOTION))
                face_img = face_img.astype('float32') / 255.0 # pre-processing 
                face_img = face_img - 0.5 # pre-processing specific to the emotion classifier
                face_img = face_img * 2.0 # pre-processing specific to the emotion classifier
                face_img = np.expand_dims(face_img, 0) # batch of one
                face_img = np.expand_dims(face_img, -1) # pre-processing specific to the emotion classifier
                probas = emotion_classifier.predict(face_img)
                emotions[j] = probas
            highest = np.where(np.mean(emotions, axis=0) == np.amax(np.mean(emotions, axis=0)))
            emotion = emo_labels[highest[0][0]]

        # Number of persons (what was in the manual)
        color_image = load_image_from_path(''.join(('movie_posters/',i)), target_size=(INPUT_SHAPE), color_mode='rgb').astype('uint8')
        color_image_batched = np.expand_dims(color_image, axis=0) # batch size of one
        R = hub_model(color_image_batched) # input the image to the modell

        detection_scores = R['detection_scores'].numpy()[0]
        detection_clases = R['detection_classes'].numpy()[0]
        detection_bounding_box = R['detection_boxes'].numpy()[0]

        indices = np.where(detection_scores > 0.5) # only select labels with p > 0.50 
        detection_classes_sel = detection_clases[indices] # get the classes where p > 0.5
        detection_scores_sel = detection_scores[indices] # get the probabilities where p > 0.5
        detection_bb_sel = detection_bounding_box[indices] # get the bounding boxes where p > 0.5

        # number of detectable objects
        n_obj = len(detection_classes_sel)

        # number of detectable persons
        n_person = 0
        for j, label in enumerate(detection_classes_sel):
            textual_label = coco_classes[int(label)]
            if textual_label == 'person':
                n_person=+1

        # Colors
        color_image = load_image_from_path(''.join(('movie_posters/',i)), target_size=None, color_mode='rgb')
        img = Image.fromarray(color_image.astype(np.uint8)) # convert to PIL image object
        colors = colorgram.extract(img, 10) 

        main_color = get_colour_name(tuple(colors[0].rgb))
        second_color = get_colour_name(tuple(colors[1].rgb))

        median_r, median_g, median_b = np.median(color_image, axis=(0,1))

        color_image_hsv = cv2.cvtColor(color_image, cv2.COLOR_RGB2HSV)
        median_h, median_s, median_v = np.median(color_image_hsv, axis=(0,1))

        # put all results in a dictionary and add that to the final results
        res = {'Path': i,'Title': imdb_info['Title'].values[0],'Genre': imdb_info['Genre'].values[0],
               'IMDB Rating':imdb_info['IMDB Score'].values[0],
               '# faces': n_faces,'# persons':n_person, '# objects':n_obj,
               'avgGender': gender, 'avgEmotion': emotion,
               'mainColor': main_color, '2ndColor': second_color, 'medianR': median_r, 'medianG': median_g,
               'medianB': median_b, 'medianH': median_h, 'medianS': median_s, 'medianV': median_v}
        results.append(res)
    except: 
        print(i) # print the images for which the algorithm doesn't work and are excluded form the results.
        
df = pd.DataFrame(results)

100534.jpg
102138.jpg
103239.jpg
1068646.jpg
1078899.jpg
111271.jpg
112844.jpg
113188.jpg
113421.jpg
119239.jpg
1217426.jpg
125209.jpg
1286130.jpg
132893.jpg
1340418.jpg
1381404.jpg
1448762.jpg
145681.jpg
1567437.jpg
1746153.jpg
1975269.jpg
2024506.jpg
2039393.jpg
209475.jpg
214965.jpg
2175671.jpg
2316801.jpg
2400283.jpg
240944.jpg
2442526.jpg
2464812.jpg
24844.jpg
251127.jpg
2557848.jpg
2649522.jpg
2752200.jpg
284769.jpg
298296.jpg
303017.jpg
3077214.jpg
318292.jpg
3266948.jpg
32846.jpg
3400980.jpg
349686.jpg
35157.jpg
3582088.jpg
365043.jpg
371746.jpg
379889.jpg
380798.jpg
385013.jpg
407612.jpg
42383.jpg
43587.jpg
444781.jpg
45109.jpg
473490.jpg
47435.jpg
4744086.jpg
477877.jpg
4871120.jpg
62456.jpg
64606.jpg
64699.jpg
65494.jpg
775362.jpg
77610.jpg
81417.jpg
82763.jpg
82912.jpg
860866.jpg
86355.jpg
866439.jpg
86723.jpg
867270.jpg
891527.jpg
91642.jpg
918557.jpg
92978.jpg
94947.jpg
959858.jpg
96184.jpg
98444.jpg
98520.jpg
99575.jpg
99731.jpg


The models I tried were had quite a bad performance, as the maximum R was 0.06. Maybe I should try making a mixed model (as we did for the epidemology course). But I think that is out of the scope of this excercise. Also, the models of this notebook aren't that good at predicting the amount of persons and faces etc. on movie posters. Better data and algorithms can improve the model. 

The best inidividual predictive feature seems to be the second color and after that the main color of the poster.

In [34]:
df

,Path,Title,Genre,IMDB Rating,# faces,# persons,# objects,avgGender,avgEmotion,mainColor,2ndColor,medianR,medianG,medianB,medianH,medianS,medianV
0,100608.jpg,The Shrimp on the Barbie (1990),Comedy,5.7,0,1,2,unknown,unknown,silver,silver,254.0,242.0,135.0,54.468086,0.450980,254.0
1,101529.jpg,Cabeza de Vaca (1991),Adventure|Drama|History,7.0,0,1,4,unknown,unknown,silver,silver,184.0,192.0,194.0,192.413788,0.126904,203.0
2,101697.jpg,Deep Blues (1992),Documentary|Music,7.3,0,1,2,unknown,unknown,white,black,123.0,148.0,144.0,161.052628,0.149573,152.0
3,102250.jpg,L.A. Story (1991),Comedy|Drama|Fantasy,6.7,0,1,2,unknown,unknown,white,maroon,247.0,245.0,238.0,60.000000,0.032389,247.0
4,1022606.jpg,Leonera (2008),Drama,7.1,0,1,6,unknown,unknown,black,gray,67.0,67.0,49.0,51.428574,0.301471,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,98105.jpg,Police Academy 6: City Under Siege (1989),Comedy|Crime,4.1,0,1,6,unknown,unknown,white,black,226.0,161.0,143.0,30.000000,0.156863,228.0
381,986230.jpg,French Film (2008),Comedy|Romance,6.5,0,1,3,unknown,unknown,white,black,206.0,182.0,169.0,0.000000,0.000000,206.0
382,988849.jpg,Donkey Punch (2008),Crime|Drama|Horror,5.2,0,1,7,unknown,unknown,white,white,196.0,207.0,179.0,170.666672,0.292517,213.0
383,993779.jpg,Ricky Gervais Live 3: Fame (2007),Comedy,7.8,0,1,1,unknown,unknown,silver,white,144.0,174.0,213.0,212.727264,0.243802,219.0
